## 資料合併-短程

In [4]:
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# 1. 合併多個 CSV 檔案
# --------------------------
def merge_files(file_paths):
    """讀取並合併多個 CSV 檔案，返回合併後的 DataFrame"""
    data_frames = []
    for file_path in file_paths:
        if os.path.exists(file_path):
            print(f"正在讀取檔案：{file_path}")
            df = pd.read_csv(file_path)
            if not df.empty:
                data_frames.append(df)
            else:
                print(f"檔案 {file_path} 為空，跳過。")
        else:
            print(f"檔案 {file_path} 不存在，跳過。")
    if data_frames:
        merged_data = pd.concat(data_frames, ignore_index=True)
        print("合併完成。")
        return merged_data
    else:
        print("沒有任何資料可供合併。")
        return pd.DataFrame()

# 設定檔案路徑
base_path = '/Users/yuchingchen/Documents/專題/cleaned_data/data/short'
file_paths = [
    f'{base_path}/tokyo.csv',
    f'{base_path}/tokyo_business.csv',
    f'{base_path}/hongkong.csv',
    f'{base_path}/hongkong_business.csv',
    f'{base_path}/singapore.csv',
    # f'{base_path}/singapore_business.csv',
    f'{base_path}/bangkok.csv',
    f'{base_path}/seoul.csv',
    f'{base_path}/seoul_business.csv'
]

# 取得合併後的資料，不存為中間檔
data = merge_files(file_paths)

# 若合併結果為空，則結束程式
if data.empty:
    exit("合併資料為空，程式結束。")

# --------------------------
# 2. 統一出發日期格式
# --------------------------
data["出發日期"] = pd.to_datetime(
    data["出發日期"], 
    format="%Y-%m-%d", 
    errors="coerce"
).combine_first(
    pd.to_datetime(data["出發日期"], format="%Y/%m/%d", errors="coerce")
)
data["出發日期"] = data["出發日期"].dt.strftime("%Y-%m-%d")

# --------------------------
# 3. 計算對數變數並移除缺失資料
# --------------------------
for col in ["平均價格", "最低價格", "價格變異", "中位數價格", "最低價格剩餘天數"]:
    log_col = col + "_log"
    data[log_col] = data[col].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan).round(2)

# 移除包含 NaN 的關鍵欄位
required_cols = ["平均價格_log", "最低價格_log", "最低價格剩餘天數_log", "價格變異_log", "中位數價格_log"]
data = data.dropna(subset=required_cols)

# 顯示抵達機場代號種類及數量
print(data['抵達機場代號'].value_counts())

# --------------------------
# 4. 假期判斷
# --------------------------
holidays = {
    "台灣": [("2025-01-20", "2025-02-02"), ("2025-02-28", "2025-03-02")],
    "日本": [("2025-02-11", "2025-02-11"), ("2025-02-22", "2025-02-24"), ("2025-03-20", "2025-03-20")],
    "香港": [("2025-01-29", "2025-01-31")],
    "新加坡": [("2025-01-29", "2025-01-30")],
    "韓國": [("2025-01-28", "2025-01-30"), ("2025-03-01", "2025-03-03")]
}

airport_to_region = {
    "HKG": "香港",
    "NRT": "日本",
    "HND": "日本",
    "BKK": "泰國",  # 泰國沒有假期，不處理
    "ICN": "韓國",
    "GMP": "韓國",
    "SIN": "新加坡"
}

# 將「出發日期」轉回 datetime 格式以利比較
data["出發日期"] = pd.to_datetime(data["出發日期"])

def is_holiday(row):
    # 台灣假期判斷（以出發地為台灣）
    for start, end in holidays["台灣"]:
        if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
            return 1
    # 根據抵達機場對應地區判斷假期
    region = airport_to_region.get(row["抵達機場代號"])
    if region and region in holidays:
        for start, end in holidays[region]:
            if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
                return 1
    return 0

data["假期"] = data.apply(is_holiday, axis=1)

# 若有「艙等」欄位，將「假期」欄位移至「艙等」欄位之後
cols = data.columns.tolist()
if '假期' in cols and '艙等' in cols:
    cols.insert(cols.index('艙等') + 1, cols.pop(cols.index('假期')))
data = data[cols]

# --------------------------
# 5. 新增額外的地區經濟資料
# --------------------------
additional_data = {
    "Region": ["香港", "日本", "新加坡", "韓國", "泰國"],
    "經濟指標": [0.6313249, -0.84717, 2.1585612, -0.059982, -1.882734]
}
region_df = pd.DataFrame(additional_data)

data["Region"] = data["抵達機場代號"].map(airport_to_region)
data = data.merge(region_df, how="left", on="Region")

# --------------------------
# 6. 新增是否為平日欄位
# --------------------------
weekend_days = ["週六", "週日"]
data["是否為平日"] = data["星期"].apply(lambda x: 1 if x in weekend_days else 0)

# 將「是否為平日」欄位移至「是否過夜」欄位之後
cols = data.columns.tolist()
if '是否過夜' in cols and '是否為平日' in cols:
    idx = cols.index("是否過夜") + 1
    cols.insert(idx, cols.pop(cols.index("是否為平日")))
data = data[cols]

# --------------------------
# 7. 新增機場指標欄位
# --------------------------
airport_index = {
    "HND": 0.7560275,
    "ICN": 1.5913391,
    "SIN": 1.0416285,
    "BKK": 0.4139873,
    "HKG": -0.229905,
    "NRT": -1.180738,
    "GMP": -2.39234
}
data['機場指標'] = data['抵達機場代號'].map(airport_index)

# 將「機場指標」移至「經濟指標」欄位之後
cols = data.columns.tolist()
if '經濟指標' in cols and '機場指標' in cols:
    flight_idx = cols.index('經濟指標')
    cols.insert(flight_idx + 1, cols.pop(cols.index('機場指標')))
data = data[cols]

# --------------------------
# 8. 新增「飛行時間_分鐘」欄位
# --------------------------
def convert_flight_time_to_minutes(time_str):
    """
    將 'x 小時 y 分鐘' 格式的字串轉換為分鐘數
    """
    if isinstance(time_str, str):
        match = re.match(r'(\d+)\s*小時(?:\s*(\d+)\s*分鐘)?', time_str)
        if match:
            hours = int(match.group(1))
            minutes = int(match.group(2)) if match.group(2) else 0
            return hours * 60 + minutes
    return None

data['飛行時間_分鐘'] = data['飛行時間'].apply(convert_flight_time_to_minutes)

# 將「飛行時間_分鐘」移至「飛行時間」欄位之後
cols = data.columns.tolist()
if '飛行時間' in cols and '飛行時間_分鐘' in cols:
    flight_idx = cols.index('飛行時間')
    cols.insert(flight_idx + 1, cols.pop(cols.index('飛行時間_分鐘')))
data = data[cols]

# --------------------------
# 9. 新增競爭航班數欄位
# --------------------------
# 依據「抵達機場代號」、「出發日期」、「出發時段」計算每一組航班數
data['competing_flights'] = data.groupby(
    ['抵達機場代號', '出發日期', '出發時段']
)['航空公司'].transform('size')

data = data.sort_values(
    by=['抵達機場代號', '出發日期', '出發時段', '航空公司']
)

# --------------------------
# 10. 移除不必要的欄位
# --------------------------
data = data.drop(columns=['查詢日期', '最低價格出現日期'])

# --------------------------
# 11. 輸出最終結果
# --------------------------
# 按出發日期與抵達機場代號排序
data = data.sort_values(by=["出發日期", "抵達機場代號"])

output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/short_flight.csv'
data.to_csv(output_path, index=False)
print(f"最終處理後的資料已儲存至：{output_path}")

正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/tokyo.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/tokyo_business.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/hongkong.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/hongkong_business.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/singapore.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/bangkok.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/seoul.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/seoul_business.csv
合併完成。
抵達機場代號
HKG    3381
NRT    2026
BKK    1596
ICN    1149
HND    1058
SIN    1053
GMP     164
Name: count, dtype: int64
最終處理後的資料已儲存至：/Users/yuchingchen/Documents/專題/cleaned_data/short_flight.csv
